[View in Colaboratory](https://colab.research.google.com/github/sayakgis/Handwritten_Digit_Prediction_DL/blob/master/DNN_Lidar_Point_Cloud.ipynb)

# Deep Learning Model to Classify Lidar Point cloud

**Install LASPY libraries: las file handler in python**

In [0]:
!pip install laspy --quiet   #quiet installation will not throw message

**Connect Google drive, we have stored the data in google drive**

This will ask for one time token code to acees data from google drive, google account sign-in is required.
Click on the link and get the token, copy and paste the token here

In [4]:
#Set-up google drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


**Mount the google storage as drive here**

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

**Import required packages**

In [0]:
from laspy.file import File
import laspy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout

**Lets check the content of folder**

In [127]:
file_list=!ls drive/Data_Deep_Learning/Lidar
file_list


['5480_2945.las', '5480_2950.las', '5480_2960.las', '5480_3170.las']

In [0]:
import glob
las_files=glob.glob('drive/Data_Deep_Learning/Lidar/*las')

classification=[]
count_returns,return_num=[],[]
intensity=[]
x,y,z=[],[],[]

for las_file in las_files:
  fh=File(las_file,mode='r')
  classification.append(fh.classification)
  count_returns.append(fh.num_returns)
  return_num.append(fh.return_num)
  intensity.append(fh.intensity)
  x.append(fh.x)
  y.append(fh.y)
  z.append(fh.z)
  df_train=pd.DataFrame({'class':classification[0]})

In [140]:
df_train.shape

(8073627, 1)

In [136]:
classification[1].shape

(6197601,)

test

In [0]:
import glob
las_files=glob.glob('drive/Data_Deep_Learning/Lidar/*las')

classification=[]
count_returns,return_num=[],[]
intensity=[]
x,y,z=[],[],[]

for las_file in las_files:
  fh=File(las_file,mode='r')
  df_train=pd.DataFrame({'class':fh.classification})

In [126]:
df_train.shape

(5037028, 1)

**Load the LAS files**

In [0]:
train_5480_2945=File('drive/Data_Deep_Learning/Lidar/5480_2945.las',mode='r')
train_5480_2950=File('drive/Data_Deep_Learning/Lidar/5480_2950.las',mode='r')
train_5480_2960=File('drive/Data_Deep_Learning/Lidar/5480_2960.las',mode='r')
train_5480_3170=File('drive/Data_Deep_Learning/Lidar/5480_3170.las',mode='r')

In [0]:
df1=pd.DataFrame({'returns':train_5480_2945.num_returns,
                  'class':train_5480_2945.classification,
                  'intensity':train_5480_2945.intensity,
                  'z':train_5480_2945.z,
                  'x':train_5480_2945.x,'y':train_5480_2945.y,
                  'return_count':train_5480_2945.return_num})

df2=pd.DataFrame({'returns':train_5480_2950.num_returns,
                  'class':train_5480_2950.classification,
                  'intensity':train_5480_2950.intensity,
                  'z':train_5480_2950.z,
                  'x':train_5480_2950.x,'y':train_5480_2950.y,
                  'return_count':train_5480_2950.return_num})
df3=pd.DataFrame({'returns':train_5480_2960.num_returns,
                  'class':train_5480_2960.classification,
                  'intensity':train_5480_2960.intensity,
                  'z':train_5480_2960.z,
                  'x':train_5480_2960.x,'y':train_5480_2960.y,
                  'return_count':train_5480_2960.return_num})
df4=pd.DataFrame({'returns':train_5480_3170.num_returns,
                  'class':train_5480_3170.classification,
                  'intensity':train_5480_3170.intensity,
                  'z':train_5480_3170.z,
                  'x':train_5480_3170.x,'y':train_5480_3170.y,
                  'return_count':train_5480_3170.return_num})

In [0]:
df=pd.concat([df1,df2,df3,df4],0)

In [147]:
df.shape

(24500250, 7)

In [148]:
df.pivot_table(values='returns',index='class')

,returns
class,
2,1.419535
4,2.045255
6,1.158790
7,1.635375
8,3.431263
17,1.424756
18,2.045242


In [0]:
y=df1['class']
X=df1.drop('class',axis=1)
ss=StandardScaler()
ss.fit(X)
X_std=ss.transform(X)

In [150]:
y_hot=pd.get_dummies(y)
y_hot.head()

,2,4,6,7,8,17,18
0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0


In [0]:
Xtrain, Xtest, ytrain, ytest=train_test_split(X_std,y_hot,random_state=123)

In [152]:
print(Xtrain.shape)
print(Xtest.shape)
print(X.shape)

(6055220, 6)
(2018407, 6)
(8073627, 6)


### Build the graph

In [0]:
model=Sequential()

In [0]:
model.add(Dense(300,'relu',input_shape=(Xtrain.shape[1],)))

model.add(Dense(200,'relu'))
model.add(Dropout(.25))
model.add(Dense(100,'relu'))
model.add(Dropout(.20))
model.add(Dense(60,'relu'))
model.add(Dropout(.10))

model.add(Dense(ytrain.shape[1],activation='softmax'))

In [0]:
model.compile('adam','categorical_crossentropy',metrics=['accuracy'])

### Execute the graph

In [161]:
model.fit(Xtrain,ytrain.values,batch_size=2000,epochs=20,validation_data=(Xtest,ytest.values))

Train on 6055220 samples, validate on 2018407 samples
Epoch 1/20
6055220/6055220 [==============================] - 36s 6us/step - loss: 0.4786 - acc: 0.8147 - val_loss: 0.4060 - val_acc: 0.8353
Epoch 2/20
 232000/6055220 [>.............................] - ETA: 37s - loss: 0.4218 - acc: 0.8303

6055220/6055220 [==============================] - 35s 6us/step - loss: 0.4124 - acc: 0.8332 - val_loss: 0.3892 - val_acc: 0.8401
Epoch 3/20
2112000/6055220 [=========>....................] - ETA: 21s - loss: 0.4022 - acc: 0.8363

6055220/6055220 [==============================] - 36s 6us/step - loss: 0.3996 - acc: 0.8372 - val_loss: 0.3789 - val_acc: 0.8437
Epoch 4/20
2838000/6055220 [=============>................] - ETA: 17s - loss: 0.3938 - acc: 0.8390

6055220/6055220 [==============================] - 35s 6us/step - loss: 0.3921 - acc: 0.8398 - val_loss: 0.3770 - val_acc: 0.8441
Epoch 5/20
3108000/6055220 [==============>...............] - ETA: 16s - loss: 0.3882 - acc: 0.8411

6055220/6055220 [==============================] - 35s 6us/step - loss: 0.3871 - acc: 0.8414 - val_loss: 0.3698 - val_acc: 0.8468
Epoch 6/20
3222000/6055220 [==============>...............] - ETA: 15s - loss: 0.3842 - acc: 0.8423

...

6055220/6055220 [==============================] - 35s 6us/step - loss: 0.3649 - acc: 0.8485 - val_loss: 0.3466 - val_acc: 0.8541
Epoch 17/20
3254000/6055220 [===============>..............] - ETA: 15s - loss: 0.3640 - acc: 0.8488

6055220/6055220 [==============================] - 35s 6us/step - loss: 0.3639 - acc: 0.8488 - val_loss: 0.3469 - val_acc: 0.8541
Epoch 18/20
3256000/6055220 [===============>..............] - ETA: 15s - loss: 0.3631 - acc: 0.8490

6055220/6055220 [==============================] - 35s 6us/step - loss: 0.3630 - acc: 0.8490 - val_loss: 0.3459 - val_acc: 0.8543
Epoch 19/20
3260000/6055220 [===============>..............] - ETA: 15s - loss: 0.3623 - acc: 0.8493

6055220/6055220 [==============================] - 35s 6us/step - loss: 0.3623 - acc: 0.8493 - val_loss: 0.3440 - val_acc: 0.8549
Epoch 20/20
3262000/6055220 [===============>..............] - ETA: 15s - loss: 0.3609 - acc: 0.8497

6055220/6055220 [==============================] - 35s 6us/step - loss: 0.3613 - acc: 0.8496 - val_loss: 0.3445 - val_acc: 0.8547
